<a href="https://colab.research.google.com/github/jkc1-4/python-for-micro/blob/main/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install tensorflow tensorflow-gpu matplotlib tensorflow-datasets ipywidgets


In [ ]:
!pip install matplotlib

In [ ]:
!pip install tensorflow-datasets

In [ ]:
!pip install ipywidgets

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip list

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu  in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)


In [ ]:
gpus

In [ ]:
#bring the tensorlfow dATAset and fashin mnist
import tensorflow_datasets as tfds
#bring the matplotlib for visual stuff
from matplotlib import pyplot as plt

In [ ]:
ds = tfds.load('fashion_mnist',split='train')

In [ ]:
ds

In [ ]:
ds.as_numpy_iterator().next()

2. viz the dataset and build dataset


In [ ]:
#do some data transformastion
import numpy as np
#setup connection aka iterator so that we can iterated and bring back the next item in the list just like as linked list in java
dataiterator = ds.as_numpy_iterator()

In [ ]:
#now we can the call the .next() to bring tha array of datasets in list using
#getting data out of a pipeline
dataiterator.next()

In [ ]:
#set up the subplot formation using matplotlib and i want to total subplot or fig to 20x20 size
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
#loop for 4 times and get images
for idx in range(4):
  #grab an image and label
  sample = dataiterator.next()
  #plot the image using the specific subplot/axis where ax is axis , squeeze means to sqeeze the co-ordinates for viz
  ax[idx].imshow(np.squeeze(sample['image']))
  #title is optional , its for the top numbering or name of images
  ax[idx].title.set_text(sample['label'])

In [ ]:
#Scale and return the images only
def scale_images(data):
  image=data['image']
  return image/255

In [ ]:
#reloading the dataset
ds = tfds.load('fashion_mnist',split='train')
#running the dataset through the scale image preprocessing step
ds = ds.map(scale_images)
#cachee the dataset for that batch
ds= ds.cache()
#shuffle it up
ds  = ds.shuffle(60000)
#batch into 128 images per sample
ds = ds.batch(128)
#reduce the likelyhood of bottlenecking
ds = ds.prefetch(64)

ds.as_numpy_iterator().next().shape


3.build the neural network
*   importing the model components
*   build the generator
*   build the discriminator


In [ ]:
#bringing in the sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
#bringin in the layers for the neural networks
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Reshape,LeakyReLU,Dropout,UpSampling2D

In [ ]:
def build_generator():
  model = Sequential()

#here we r specifing what our input layer is going to be ,i.e, a dense fully connect layer, 7,7,128 units means the 128 image input is going to convert to 7*7*128 spacial quality
# in conditional GAN we can specify what image we want to generate but here it is generating random images from sample population
  model.add(Dense(7*7*128, input_dim=128))
  #this just means y=0.2 how thw graph is slope
  model.add(LeakyReLU(0.2))
  #now we convert the dense layer into the begining of a image of size 7*7*128 which inventually will be 28*28*1
  model.add(Reshape((7,7,128)))

  #upsampling block 1
  model.add(UpSampling2D()) # this doubles the size of output
  model.add(Conv2D(128,5, padding='same'))
  model.add(LeakyReLU(0.2))

  #upsampling block 2
  model.add(UpSampling2D()) # this doubles the size of output
  model.add(Conv2D(128,5, padding='same'))
  model.add(LeakyReLU(0.2))

  #convolutional block 1
  model.add(Conv2D(128,4 , padding = 'same'))
  model.add(LeakyReLU(0.2))

  #con block 2
  model.add(Conv2D(128,4,padding = 'same'))
  model.add(LeakyReLU(0.2))

  #Conc layer to get to one channel
  model.add(Conv2D(1,4,padding='same' ,activation = 'sigmoid'))

  return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
img = generator.predict(np.random.randn(4,128,1))
img.shape

In [ ]:
#set up the subplot formation using matplotlib and i want to total subplot or fig to 20x20 size
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
#loop for 4 times and get images
for idx ,img in enumerate(img): #this to get the index in image

  #plot the image using the specific subplot/axis where ax is axis , squeeze means to sqeeze the co-ordinates for viz
  ax[idx].imshow(np.squeeze(img))
  #title is optional , its for the top numbering or name of images
  ax[idx].title.set_text(idx)

In [ ]:
def build_discriminator():
  model = Sequential()

  #First conv block
  model.add(Conv2D(32,5,input_shape=(28,28,1)))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))#0.4 is strength of the dropout

  #second conv block
  model.add(Conv2D(64,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #thrid conv block
  model.add(Conv2D(128,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #forth conv block
  model.add(Conv2D(256,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #Flatten then passa thru dense layer
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1,activation='sigmoid'))#1 represent false img
  return model

In [ ]:
discriminator = build_discriminator()
discriminator.summary()


In [ ]:
img.shape

In [ ]:
discriminator.predict(img)

**Construct Training loop**

1.   setup loses and optimizers

In [ ]:
#adam is going to be the optimizers for both
from tensorflow.keras.optimizers import Adam
# binary cross entropy is goinhg to be the loss for both
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
g_opt=Adam(learning_rate=0.0001)
d_opt=Adam(learning_rate=0.00001)
g_loss=BinaryCrossentropy()
d_loss=BinaryCrossentropy()

**4.2 Build Subclass Model**

In [ ]:
#Importing the base model class to subclass
from tensorflow.keras.models import Model

In [ ]:
tf.random.normal((6,128,1))

In [ ]:
class FashionGAN(Model):
  def __init__(self, generator, discriminator, *args, **kwargs):
    #pass thru *args kwargs to base class
    super().__init__(*args,**kwargs)

    #create attributes  for gen and discr
    self.generator = generator
    self.discriminator = discriminator


  def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
    #compile the base class
    super().compile(*args, **kwargs)

  #create attributes for losses and optimizers
    self.g_opt=g_opt
    self.d_opt=d_opt
    self.g_loss= g_loss
    self.d_loss= d_loss

  def train_step(self, batch):
    #get the data
    real_images = batch
    fake_images = self.generator(tf.random.normal((128,128,1)),training=False)

    #Train the discriminator
    with tf.GradientTape() as d_Tape:

      #Pass the real and fake image to the discriminator model
      yhat_real= self.discriminator(real_images, training=True)
      yhat_fake = self.discriminator(fake_images, training=True)
      yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)


      #Create labels for the real and fake images
      y_realfake = tf.concat([tf.zeros_like(yhat_real),tf.ones_like(yhat_fake)], axis=0)

      #Add some noise to the TRUE outputs
      noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
      noise_fake = 0.15*tf.random.uniform(tf.shape(yhat_fake))
      y_realfake = tf.concat([noise_real,noise_fake] , axis=0)

      #Calculate the loss
      total_d_loss = self.d_loss(y_realfake, yhat_realfake)

    #Apply the backpropogation -- this is how neural network learn
    dgrad = d_Tape.gradient(total_d_loss, self.discriminator.trainable_variables)
    self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

  #def test_step(self):


    #TRAIN THE GENERATOR
    with tf.GradientTape() as g_tape:

      #generate some new images
      gen_images = self.generator(tf.random.normal((128,128,1)),training=True)

      #create predicted labels
      predicted_labels = self.discriminator(gen_images, training=False)

      #calculate loss
      total_g_loss = self.g_loss(tf.zeros_like(predicted_labels),predicted_labels)

    #apply back-prop
    ggrad= g_tape.gradient(total_g_loss, self.generator.trainable_variables)
    self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

    return {"d_loss":total_d_loss, "g_loss":total_g_loss}


In [ ]:
#create instance of subclasses model above
fashgan = FashionGAN(generator,discriminator)


In [ ]:
fashgan.compile(g_opt,d_opt,g_loss,d_loss)

**Build callback**

In [ ]:
import os #help with photo navigation
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('images', f'generated_img_{epoch}_{i}.png'))

In [ ]:
hist=fashgan.fit(ds,epochs=20,  callbacks=[ModelMonitor()])
#recommended 2000 epochs  callbacks=[ModelMonitor()]